In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_community.chat_models import ChatOllama

# Create a ChatOpenAI model
model = ChatOllama(model="llama3.1")

In [2]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert product reviewer."),
        ("human", "List the main features of the product {product_name}."),
    ]
)


In [3]:
# Define pros analysis step
def analyze_pros(features):
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the pros of these features.",
            ),
        ]
    )
    return pros_template.format_prompt(features=features)


# Define cons analysis step
def analyze_cons(features):
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the cons of these features.",
            ),
        ]
    )
    return cons_template.format_prompt(features=features)


In [4]:
# Combine pros and cons into a final review
def combine_pros_cons(pros, cons):
    return f"Pros:\n{pros}\n\nCons:\n{cons}"

In [5]:
# Simplify branches with LCEL
pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)

In [6]:
# Create the combined chain using LangChain Expression Language (LCEL)
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={"pros": pros_branch_chain, "cons": cons_branch_chain})
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)

In [7]:
# Run the chain
result = chain.invoke({"product_name": "MacBook Pro"})

In [8]:
# Output
print(result)

Pros:
Here are the pros of the MacBook Pro's features:

**Hardware Features:**

1. **Up to 10th-generation Intel Core i9 processors (8-cores)**:
	* Fast processing speed
	* Ability to handle demanding tasks and multitasking
2. **Up to 64GB DDR4 RAM**:
	* High memory capacity for running multiple applications simultaneously
	* Reduced likelihood of memory errors or crashes
3. **Up to 8TB SSD storage**:
	* Extremely fast data access and loading times
	* High storage capacity for storing large files, programs, and databases
4. **13-inch, 15-inch, or 16-inch Retina display with high resolution (2560 x 1600 pixels) and True Tone technology**:
	* Stunning visuals and color accuracy
	* Wider viewing angles and reduced glare

**Performance Features:**

1. **Supports up to four external displays via USB-C or Thunderbolt 3 ports**:
	* Enhanced multitasking capabilities
	* Ability to work with multiple screens and applications simultaneously
2. **Up to 8x faster than the previous generation MacBo